# Uniview module to show Apollo 8 Earthrise location

*Aaron Geller, 2018*

### Imports and function definitions

In [1]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data" 
import sys, os, shutil, errno, string, urllib


sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir) )))
import uvmodlib.v1 as uvmod

### USES Conf Template 

In [39]:
Template = """mesh
{  
    data geom    ./modules/$folderName/MoonOrbit.raw

    cullRadius $cr
    glslVersion 330
    
    propertyCollection 
    {        
        __objectName__
        {           

            
        }
    }
    pass
    {
        useDataObject geom
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/shader.gs
                vertexShader ./modules/$folderName/shader.vs
                fragmentShader   ./modules/$folderName/shader.fs
                                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }

}"""

### IceCube class

In [42]:
class IceCube():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("shader.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("shader.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("shader.fs",dataFolder)

        self.cr = 1000
        self.Scale = 1

    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(Template)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir,
                                 cr = self.cr,
                                 Scale = self.Scale
                                 ))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [43]:
model = IceCube(uvmod.OrbitalObject())
generator = uvmod.Generator()
scene = uvmod.Scene()
scene.setname("Earthrise")
scene.setparent("Moon")
scene.setentrydist(100000)
scene.setunit(1)
#scene.setsurfacepositionerrotation(0,0.,-90)
#scene.setpositionfile(uvmod.PositionFileTypes.Surface("Earth",  0.0, -89., -0.2))

modinfo = uvmod.ModuleInformation()

### Specify Settings and generate the module

In [44]:
model.object.setcameraradius(10)
model.object.setcoord(scene.name)
model.object.setname("Earthrise")
model.object.setguiname("/AaronGeller/Earthrise")
model.object.settargetradius(20)
model.object.showatstartup(False)
model.cr = 10000
modinfo.setname("Earthrise")
modinfo.setauthor("Aaron Geller<sup>1</sup> (1)Adler Planetarium")
modinfo.cleardependencies()
modinfo.setdesc("Uniview module for Apollo 8 Earthrise location")
modinfo.setversion("1.0")

generator.generate("Earthrise",[scene],[model],modinfo)
uvmod.Utility.senduvcommand(model.object.name+".reload")

## Helper Functions for modifing code
*Reload Module and Shaders in Uniview*

In [9]:
uvmod.Utility.senduvcommand(model.object.name+".reload; system.reloadallshaders")

Unable to connect to Uniview


*Copy modified Shader files and reload*

In [30]:
from config import Settings

In [37]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+model.object.name)
uvmod.Utility.senduvcommand(model.object.name+".reload")